In [8]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
# from torch.utils.data import Dataset, DataLoader


In [3]:
dataset=pd.read_csv('EncodedFinalDataset.csv')

In [5]:
dataset.head(3)

,Price,Bedrooms,Bathrooms,Land_Area,Year,Status_For Rent,Status_For Sale,Furnished_No,Furnished_Yes,Address_Addis Ketema,...,Address_Bole,Address_Gullele,Address_Kirkos,Address_Kolfe Keranio,Address_Lemi Kura,Address_Lideta,Address_Nifas Silk-Lafto,Address_Yeka,Property_Type_Apartment,Property_Type_Villa
0,70000.0,1.0,2.0,148.0,2025.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,100000.0,3.0,2.0,148.0,2025.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,31000000.0,5.0,3.0,350.0,2025.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [6]:
dataset.shape

(7659, 22)

In [ ]:
X=dataset[1:]
Y=dataset[0]


In [20]:
dataset=torch.tensor(dataset.values,dtype=torch.float32)
dataset.type

<function Tensor.type>

In [36]:
X=dataset[:,1:]
Y=dataset[:,0]
Y=Y.view(-1,1)


In [37]:
X.shape,Y.shape

(torch.Size([7659, 21]), torch.Size([7659, 1]))

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape